In [ ]:
!git clone https://github.com/Varuns-github/emotion-based-music-generator.git

Cloning into 'emotion-based-music-generator'...
remote: Enumerating objects: 300, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 300 (delta 4), reused 41 (delta 3), pack-reused 255
Receiving objects: 100% (300/300), 182.00 MiB | 11.93 MiB/s, done.
Resolving deltas: 100% (16/16), done.
Checking out files: 100% (234/234), done.


In [ ]:
!ls

emotion-based-music-generator  sample_data


In [ ]:
!ffmpeg -i emotion-based-music-generator/MusicDataset/Angry/121.mp3 -ab 320k -ac 2 -ar 44100 -vn 1.wav

ffmpeg version 3.4.11-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-li

In [ ]:
!mkdir angry

In [ ]:
import os

test = os.listdir("emotion-based-music-generator/MusicDataset/Angry")

for i in test:
  
  oldPath = "emotion-based-music-generator/MusicDataset/Angry/" + i
  newPath = "angry/" + i + ".wav"
  print(oldPath)
  !ffmpeg -i $oldPath -ab 320k -ac 2 -ar 44100 -vn $newPath

emotion-based-music-generator/MusicDataset/Angry/135.mp3
ffmpeg version 3.4.11-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --ena

In [ ]:
from scipy.io.wavfile import read, write

rate, music1 = read('angry/121.mp3.wav')

rate, music2 = read('angry/122.mp3.wav')

In [ ]:
import pandas as pd

music1 = pd.DataFrame(music1)
music2 = pd.DataFrame(music2)

In [ ]:
music1

,0,1
0,0,0
1,-1,0
2,0,0
3,0,0
4,0,0
...,...,...
399995,876,6727
399996,1153,6645
399997,1498,6588
399998,2158,6652


In [ ]:
import numpy as np

def create_train_dataset(df, look_back, train=True):
    dataX1, dataX2 , dataY1 , dataY2 = [],[],[],[]
    for i in range(len(df)-look_back-1):
        dataX1.append(df.iloc[i : i + look_back, 0].values)
        dataX2.append(df.iloc[i : i + look_back, 1].values)
        if train:
            dataY1.append(df.iloc[i + look_back, 0])
            dataY2.append(df.iloc[i + look_back, 1])
    if train:
        return np.array(dataX1), np.array(dataX2), np.array(dataY1), np.array(dataY2)
    else:
        return np.array(dataX1), np.array(dataX2)

In [ ]:
X1, X2, y1, y2 = create_train_dataset(pd.concat([music1.iloc[0:160000, :],music2.iloc[0:160000, :]], axis=0), look_back=3, train=True)

In [ ]:

# function to create train data by shifting the music data
def create_test_data(df, look_back):
    dataX1, dataX2 = [], []
    for i in range(len(df)-look_back-1):
        dataX1.append(df.iloc[i : i + look_back, 0].values)
        dataX2.append(df.iloc[i : i + look_back, 1].values)
        
    return np.array(dataX1), np.array(dataX2)

In [ ]:
test1, test2 = create_test_data(pd.concat([music1.iloc[160001 : 400000, :],music2.iloc[160001 : 400000, :]], axis=0), 3)

In [ ]:
X1 = X1.reshape((-1, 1, 3))
X2 = X2.reshape((-1, 1, 3))

In [ ]:
from keras.layers import Dense, LSTM, LeakyReLU
from keras.models import Sequential, load_model

rnn1 = Sequential()
rnn1.add(LSTM(units=100, activation='linear', input_shape=(None, 3)))
rnn1.add(LeakyReLU())
rnn1.add(Dense(units=50, activation='linear'))
rnn1.add(LeakyReLU())
rnn1.add(Dense(units=25, activation='linear'))
rnn1.add(LeakyReLU())
rnn1.add(Dense(units=12, activation='linear'))
rnn1.add(LeakyReLU())
rnn1.add(Dense(units=1, activation='linear'))
rnn1.add(LeakyReLU())

rnn1.compile(optimizer='adam', loss='mean_squared_error')
rnn1.fit(X1, y1, epochs=20, batch_size=100)

Epoch 1/20
3200/3200 [==============================] - 13s 4ms/step - loss: 439236.6562
Epoch 2/20
3200/3200 [==============================] - 11s 4ms/step - loss: 65694.8281
Epoch 3/20
3200/3200 [==============================] - 11s 4ms/step - loss: 59333.3047
Epoch 4/20
3200/3200 [==============================] - 12s 4ms/step - loss: 59621.9023
Epoch 5/20
3200/3200 [==============================] - 12s 4ms/step - loss: 57412.9609
Epoch 6/20
3200/3200 [==============================] - 11s 4ms/step - loss: 56126.8711
Epoch 7/20
3200/3200 [==============================] - 12s 4ms/step - loss: 56033.2695
Epoch 8/20
3200/3200 [==============================] - 11s 4ms/step - loss: 55534.0508
Epoch 9/20
3200/3200 [==============================] - 12s 4ms/step - loss: 54824.8398
Epoch 10/20
3200/3200 [==============================] - 11s 4ms/step - loss: 54347.9414
Epoch 11/20
3200/3200 [==============================] - 12s 4ms/step - loss: 54342.8984
Epoch 12/20
3200/3200 [======

In [ ]:
# LSTM Model for channel 2 of the music data
rnn2 = Sequential()
rnn2.add(LSTM(units=100, activation='linear', input_shape=(None, 3)))
rnn2.add(LeakyReLU())
rnn2.add(Dense(units=50, activation='linear'))
rnn2.add(LeakyReLU())
rnn2.add(Dense(units=25, activation='linear'))
rnn2.add(LeakyReLU())
rnn2.add(Dense(units=12, activation='linear'))
rnn2.add(LeakyReLU())
rnn2.add(Dense(units=1, activation='linear'))
rnn2.add(LeakyReLU())

rnn2.compile(optimizer='adam', loss='mean_squared_error')
rnn2.fit(X2, y2, epochs=20, batch_size=100)

Epoch 1/20
3200/3200 [==============================] - 13s 4ms/step - loss: 482849.3125
Epoch 2/20
3200/3200 [==============================] - 12s 4ms/step - loss: 67306.5469
Epoch 3/20
3200/3200 [==============================] - 14s 4ms/step - loss: 63368.6367
Epoch 4/20
3200/3200 [==============================] - 11s 4ms/step - loss: 61744.1133
Epoch 5/20
3200/3200 [==============================] - 12s 4ms/step - loss: 62590.4648
Epoch 6/20
3200/3200 [==============================] - 12s 4ms/step - loss: 60926.8242
Epoch 7/20
3200/3200 [==============================] - 11s 4ms/step - loss: 61196.3125
Epoch 8/20
3200/3200 [==============================] - 12s 4ms/step - loss: 60607.8750
Epoch 9/20
3200/3200 [==============================] - 12s 4ms/step - loss: 60285.1680
Epoch 10/20
3200/3200 [==============================] - 11s 4ms/step - loss: 59811.3203
Epoch 11/20
3200/3200 [==============================] - 12s 4ms/step - loss: 59234.9102
Epoch 12/20
3200/3200 [======

In [ ]:
pred_rnn1 = rnn1.predict(test1.reshape(-1, 1, 3))
pred_rnn2 = rnn2.predict(test2.reshape(-1, 1, 3))

In [ ]:
write('pred_rnn.wav', rate, pd.concat([pd.DataFrame(pred_rnn1.astype('int16')), pd.DataFrame(pred_rnn2.astype('int16'))], axis=1).values)


In [ ]:
from IPython.display import Audio 

a = Audio("pred_rnn.wav")
a